## Understand the datasets

For this notebook it will be using three datasets that are available on the city of Chicago's Data Portal:

1.  Socioeconomic Indicators in Chicago
2.  Chicago Public Schools
3.  Chicago Crime Data

### 1. Socioeconomic Indicators in Chicago

    This dataset contains a selection of six socioeconomic indicators of public health 
    significance and a “hardship index,” for each Chicago community area, 
    for the years 2008 – 2012.



### 2. Chicago Public Schools

    This dataset shows all school level performance data used to create CPS School 
    Report Cards for the 2011-2012 school year. This dataset is provided by 
    the city of Chicago's Data Portal.



### 3. Chicago Crime Data

    This dataset reflects reported incidents of crime (with the exception of murders 
    where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.


In [1]:
# Connect to the database
# First load the SQL extension and establish a connection to the database
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect("SQLiteMagic.db")
cur = con.cursor()

In [3]:
%sql sqlite:///SQLiteMagic.db

In [4]:
# To analyze the data using SQL, it first needs to be stored in the database.
# We will first read the csv files from the given url into pandas dataframes
# Then converting each csv file to a table in sqlite with the csv data loaded in it.

import pandas as pd

df1 = pd.read_csv('ChicagoCensusData.csv')
df1.to_sql("census_data", con, if_exists='replace', index=False,method="multi")

df2 = pd.read_csv('ChicagoCrimeData.csv')
df2.to_sql("crime_data", con, if_exists='replace', index=False,method="multi")

df3 = pd.read_csv('ChicagoPublicSchools.csv')
df3.to_sql("public_schools_data", con, if_exists='replace', index=False)


566

In [5]:
# It can be verified that the tables were created sucessfully by making a basic query

%sql select * from census_data limit 5;

 * sqlite:///SQLiteMagic.db
Done.


AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


### Problem 1

##### Find the total number of crimes recorded in the CRIME table.

In [9]:
%sql select count(*) as Total_number_of_crimes from crime_data;

 * sqlite:///SQLiteMagic.db
Done.


Total_number_of_crimes
533


### Problem 2

##### List community areas with per capita income less than 11000.

In [10]:
%%sql 
select community_area_name, 
per_capita_income from census_data where per_capita_income < 11000;

 * sqlite:///SQLiteMagic.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


### Problem 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [11]:
%sql select case_number from crime_data where description like '%MINOR%';

 * sqlite:///SQLiteMagic.db
Done.


CASE_NUMBER
HL266884
HK238408


### Problem 4

##### List all kidnapping crimes involving a child?

In [12]:
%sql select * from crime_data where primary_type like '%KIDNAPPING%'

 * sqlite:///SQLiteMagic.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5

##### What kinds of crimes were recorded at schools?

In [13]:
%sql select DISTINCT primary_type from crime_data where location_description like '%SCHOOL%'

 * sqlite:///SQLiteMagic.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6

##### List the average safety score for each type of school.

In [14]:
%%sql 

SELECT "Elementary, Middle, or High School", 
AVG(SAFETY_SCORE) AS AVG_SCORE FROM public_schools_data GROUP BY "Elementary, Middle, or High School";  

 * sqlite:///SQLiteMagic.db
Done.


"Elementary, Middle, or High School",AVG_SCORE
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7

##### List 5 community areas with highest % of households below poverty line

In [15]:
%%sql 

select COMMUNITY_AREA_NAME, percent_households_below_poverty 
from census_data order by percent_households_below_poverty desc limit 5;

 * sqlite:///SQLiteMagic.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8

##### Which community area is most crime prone?


In [16]:
%%sql select community_area_number, count(*) as crimes_number 
from crime_data group by community_area_number order by crimes_number desc limit 1;

 * sqlite:///SQLiteMagic.db
Done.


COMMUNITY_AREA_NUMBER,crimes_number
25.0,43


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index


In [17]:
%%sql 

select community_area_name, hardship_index from census_data 
where hardship_index = (select max(hardship_index) from census_data);

 * sqlite:///SQLiteMagic.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [18]:
%%sql 

select community_area_number, count(*) as number_of_crimes, 
(select community_area_name from census_data where area_number = community_area_number) as Area_name
from crime_data group by community_area_number order by number_of_crimes desc limit 1 

 * sqlite:///SQLiteMagic.db
Done.


COMMUNITY_AREA_NUMBER,number_of_crimes,Area_name
25.0,43,Austin


In [19]:
# Close the connection
con.close()